[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/LivePortrait-jupyter/blob/main/LivePortrait_jupyter.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/LivePortrait
%cd /content/LivePortrait

!rm -rf /content/LivePortrait/pretrained_weights
!git clone https://huggingface.co/camenduru/LivePortrait /content/LivePortrait/pretrained_weights

!pip install tyro==0.8.5 onnxruntime-gpu==1.18.1 onnx==1.16.1 gradio==4.36.1 colorama

%cd /content/LivePortrait/src/utils/dependencies/insightface/thirdparty/face3d/mesh/cython
!python setup.py build_ext --inplace

In [ ]:
%cd /content/LivePortrait
!python inference.py -s /content/LivePortrait/assets/examples/source/s7.jpg -d /content/LivePortrait/assets/examples/driving/d7.mp4